<a href="https://colab.research.google.com/github/williambrunos/Introduction-To-ML/blob/main/Class_2/Class_2_2/Decision_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Árvores de decisão

Árvores de decisão são estruturas de dados baseados em "se...então" para classificar ou realizar predições (regressões) de dados numéricos. Em geral, estas árvores são baseadas em perguntas, e as respostas a essas perguntas são responsáveis por classificar a resposta.

Ademais, uma ávore de decisão não necessariamente precisa ser baseada em uma única pergunta, e as respostas podem ser dados categóricos ou numéricos.

Para cada amostra de dados, descemos na árvore desde o nó raiz até chegarmos a um ponto em que não se faz mais necessário realizar nenhuma "pergunta", e é aí em que as classificações são feitas.

## Algoritmo

O algoritmo se concentra em: 

- Dadas as características do dataset e das observações, observar o quão bem cada uma divide os dados entre os labels conhecidos. Uma divisão feita de forma não perfeita, ou seja, com resultados de labels diferentes, é chamada de divisão impura.

- Para determinar qual divisão é melhor, temos de ter um meio de calcular a impureza de cada divisão.

$$gini=1-(probability\;of\;yes)^2 - (probability\;of\;no)^2$$

- É comum que divisões baseadas em alguma tabela (característica) resultem em nós com quantidades de observações diferentes. Portanto, para calcular a impureza (gini) daquela divisão baseada na característica em questão, tomamos a quantidade de observações nos nós divida pela quantidade total de observações nos dois nós e realizamos uma média ponderada destas divisões com base nos valores de impurezas descobertos.

- A característica que possuir o menor índice de Gini é então utilizada como nó separador, podendo levar a mais divisões impuras, requisitando novamente a execução do algoritmo.

- Caso uma característica seja numérica contínua, seus valores são ordenados do menor para o maior, as médias entre valores adjacentes são obtidas e estas médias são utilizadas para calcular o gini do nó que faz "característica numérica < valor". O menor índice de gini para uma certa média faz com que tal média seja posta no lugar de "valor".

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston, load_iris
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error

Iremos utilizar uma random tree para classificação no dataset **iris** e para regressão no **boston**.

### Classificador com Random Tree

In [5]:
classifier = DecisionTreeClassifier()

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13, test_size=0.2)

In [6]:
classifier.fit(X_train, y_train)

y_preds = classifier.predict(X_test)

In [9]:
accuracy_score(y_test, y_preds)

0.9666666666666667

In [12]:
print(classification_report(y_test, y_preds)) # um modelo muito bom (dataset simples)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       0.89      1.00      0.94         8
           2       1.00      0.92      0.96        13

    accuracy                           0.97        30
   macro avg       0.96      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



Perceba que uma árvore de decisão possui determinada profundidade, que por padrão do scikit-learn é automática até que se obtenham nós puros. Isto pode ser um fator prejudicial, pois temos então uma grande chance de obter overfitting de dados. 

Por conta disso, fazemos um procedimento chamado de "poda", restrigindo a profundidade máxima da árvore de modo que a mesma aprenda apenas as características gerais dos dados de treino.

In [16]:
def score_classifier(X_train, X_test, y_train, y_test, max_depth=5):
  classifier = DecisionTreeClassifier(max_depth=max_depth)
  classifier.fit(X_train, y_train)
  return classifier.score(X_test, y_test)

In [17]:
max_depths = [2, 5, 7, 9, 11, 13, 15]

for max_depth in max_depths:
  score = score_classifier(X_train, X_test, y_train, y_test, max_depth)
  print(f'Score: {score} for depht {max_depth}') # best depth -> 7 or 9 -> reduces overfitting

Score: 0.9666666666666667 for depht 2
Score: 0.9333333333333333 for depht 5
Score: 0.9333333333333333 for depht 7
Score: 0.9333333333333333 for depht 9
Score: 1.0 for depht 11
Score: 0.9666666666666667 for depht 13
Score: 0.9666666666666667 for depht 15


### Regressor com Random Tree

In [18]:
X, y = load_boston(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13, test_size=13)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

In [21]:
regressor = DecisionTreeRegressor()
regressor.fit(X_train, y_train)

y_preds = regressor.predict(X_test)
mae = mean_absolute_error(y_test, y_preds)
print(mae)

4.392307692307693


In [22]:
def score_regression(X_train, X_test, y_train, y_test, max_depth=5):
  regressor = DecisionTreeRegressor(max_depth=max_depth)
  regressor.fit(X_train, y_train)
  y_preds = regressor.predict(X_test)
  mae = mean_absolute_error(y_test, y_preds)
  return mae

In [23]:
for max_depth in max_depths:
  mae = score_regression(X_train, X_test, y_train, y_test, max_depth)
  print(f'Max depth {max_depth} produces MAE {mae}') # -> best max depth: 2 -> reduces underfitting

Max depth 2 produces MAE 2.79961753353119
Max depth 5 produces MAE 3.4420449177051973
Max depth 7 produces MAE 3.4877153188691636
Max depth 9 produces MAE 2.9280490890688267
Max depth 11 produces MAE 3.726768103691181
Max depth 13 produces MAE 4.0628205128205135
Max depth 15 produces MAE 3.596153846153847


### Feature Importances

Podemos utilizar o atributo **feature_importances_** das decision trees para termos noção do quanto cada atributo (coluna do dataset) afeta a predição do modelo. Não é interessante retirar todas as características menos influentes de uma vez, e sim aos poucos, pois muitas características podem possuir certa correlação, podendo afetar mais as predições do modelo após alguma outra ser retirada.

In [24]:
from sklearn.datasets import load_breast_cancer

In [25]:
X, y = load_breast_cancer(return_X_y=True)

In [27]:
model = DecisionTreeClassifier()
model.fit(X, y)

DecisionTreeClassifier()

In [28]:
model.feature_importances_

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.00689159, 0.00877112, 0.03747995, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00204521, 0.00100384,
       0.        , 0.        , 0.00563858, 0.        , 0.        ,
       0.69559352, 0.08856128, 0.        , 0.0110859 , 0.01440488,
       0.        , 0.01411125, 0.10709688, 0.        , 0.007316  ])